In [19]:
from bs4 import BeautifulSoup as bs
from slugify import slugify
import requests
import json
import os

In [20]:
if not os.path.exists("Dungeons"):
    os.makedirs("Dungeons")
if not os.path.exists("Dungeons/txt"):
    os.makedirs("Dungeons/txt")

## Get the index page of all dungeons

In [21]:
URL = "https://www.dofuspourlesnoobs.com/donjons.html"
base_URL = "https://www.dofuspourlesnoobs.com"
dungeon_index_page = requests.get(URL)
dungeon_index_soup = bs(dungeon_index_page.content, "html.parser")

### Get all dungeon links

In [22]:
dungeon_info_list = {}

dungeon_table = dungeon_index_soup.find("table", id="trier")
dungeon_list = dungeon_table.find("tbody")
for dungeon_elem in dungeon_list.find_all("tr"):
    dungeon_info = dungeon_elem.find_all("td")
    dungeon_info_list[dungeon_info[1].find("a").text.strip()] = {"link": dungeon_info[1].find("a")["href"], "lvl": dungeon_info[-1].text.strip()}

In [23]:
print(dungeon_info_list["Aquadôme de Merkator"])

{'link': '/aquadocircme-de-merkator.html', 'lvl': '200'}


In [24]:
with open("Dungeons/list.json", "w", encoding="utf-8")as file:
        json.dump(dungeon_info_list, file)

## Download all pages

In [25]:
for key in dungeon_info_list.keys():
    dungeon = dungeon_info_list[key]
    page_url = f"{base_URL}{dungeon["link"]}"
    page_soup = bs(requests.get(page_url).content).find("div", id="wsite-content")
    for div in page_soup:
        string = str(div)
        if "PUBLICITE" in string or "data-ad-text" in string:
            div.decompose()
    with open("Dungeons" + dungeon["link"], "w", encoding="utf-8")as file:
        file.write(str(page_soup))
    with open(("Dungeons/txt" + dungeon["link"].replace("/.html", ".txt")), "w", encoding="utf-8")as file:
        file.write(page_soup.get_text())